# Fine Tuning Transformer for MultiLabel Text Classification

### **Вводная часть**

- В этом ноуте мы применим подход fine-tune для трансформерной модели BERT
- Данная задача является задачей multilabel text classification (много меточная классификация)
- Модели предстоит классифицировать текст в одну или несколько категории из списка (например фильм может быть классифицирован в одну или несколько жанров)

### **Метки**

На вход в модель передается список принадлежности документа к каждому в списе категории (указанные ниже)

Категории классификации
 - `toxic`
 - `severe_toxic`
 - `obscene`
 - `threat`
 - `insult`
 - `identity_hate`

Если комментарии размечен как <code>toxic</code> и <code>obscene</code> то для этих двух меток у нас ставится 1, а все остальные 0

Разница между Multiclass/Multilabel Classification:
- Мы рассматриваем каждую категорию отдельно друг от друга (ie. вероятность принадлежности к данной категории)
- Для функции потерь воспользуемся BCE (nn.BCELoss)
- Соответсвенно будем использовать Sigmoid, а не Softmax

In [7]:
!pip install -q transformers

In [8]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from warnings import filterwarnings; filterwarnings('ignore')

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05

### **Данные**
- В данном датасете у нас 159т примеров
- Метка является списком принадлежности некому классу токсичности (multilabel classification)

In [10]:
df = pd.read_csv("/kaggle/input/testingsdd/train.csv")
df['list'] = df[df.columns[2:]].values.tolist()
new_df = df[['comment_text', 'list']].copy()
print(new_df.shape)
new_df.head()

(159571, 2)


,comment_text,list
0,Explanation\nWhy the edits made under my usern...,"[0, 0, 0, 0, 0, 0]"
1,D'aww! He matches this background colour I'm s...,"[0, 0, 0, 0, 0, 0]"
2,"Hey man, I'm really not trying to edit war. It...","[0, 0, 0, 0, 0, 0]"
3,"""\nMore\nI can't make any real suggestions on ...","[0, 0, 0, 0, 0, 0]"
4,"You, sir, are my hero. Any chance you remember...","[0, 0, 0, 0, 0, 0]"


In [10]:
# Пример текста
new_df.iloc[0][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [11]:
# Пример метки
new_df.iloc[0][1]

[0, 0, 0, 0, 0, 0]

### **Подгрузка from_pretrained**

Будем использовать предобученную модель и токенизатор

In [4]:
# загружаем токенизатор
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(tokenizer)

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


In [64]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
print(tokenizer)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


In [65]:
bert_lm = AutoModel.from_pretrained('bert-base-uncased',return_dict=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### **Создаем Датасет**

- Загружаем датасет (CustomDataset) и токенизатор (BertTokenizer), кодируем токинизированный текст в числа используя (encode_plus)
- Для BERT, нам нужно подгружать <code>input_ids</code>, <code>mask</code>, <code>token_type_ids</code>



In [66]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe['comment_text']
        self.targets = dataframe['list']
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        
        # string comment
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        # encode text
        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [67]:
text = 'sample'
tokenizer.encode_plus(text,add_special_tokens=True)

{'input_ids': [101, 7099, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [88]:
# Вход в языковую модель (пример)
# метки подаются в ввиде списка, для BERT нужно ids,mask,token_type_ids

CustomDataset(train_dataset, tokenizer, MAX_LEN)[0].keys()

dict_keys(['ids', 'mask', 'token_type_ids', 'targets'])

In [89]:
CustomDataset(train_dataset, tokenizer, MAX_LEN)[0]['targets']

tensor([0., 0., 0., 0., 0., 0.])

### **Подготовка Данных**

Создаем Dataset, разбиваем та train,test и создаем DataLoader

In [68]:
# Разбиваем выборку на train/test
train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("train размер: {}".format(train_dataset.shape))
print("test размер: {}".format(test_dataset.shape))

# Создаем датасет
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

train размер: (127657, 2)
test размер: (31914, 2)


In [69]:
training_loader = DataLoader(training_set,batch_size=TRAIN_BATCH_SIZE,shuffle=True,num_workers=0)
testing_loader = DataLoader(testing_set,batch_size=VALID_BATCH_SIZE,shuffle=True,num_workers=0)

### **LM BERT + Классификация Head**

Будем использовать BERT для задачи классификации multilabel классификации
- Создадим три слоя; BERT, регуляризация и линейное преобразование от 768 до 6

In [70]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = bert_lm
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, 
                             attention_mask = mask, 
                             token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [71]:
# Функция потерь и оптимизатор
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), 
                             lr=LEARNING_RATE)

### **Fine-Tune BERT**

Дообучаем BERT на нашем датасете train, для дообучения не требуется много итерации

In [73]:
# Тренировачный цикл
def train(epoch):
    
    model.train()
    for _,data in enumerate(training_loader, 0):
        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        # модель требует ids,mark + token_type
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  0.7660607695579529
Epoch: 0, Loss:  0.1004597544670105
Epoch: 0, Loss:  0.1402251422405243
Epoch: 0, Loss:  0.003868764266371727


### **Валидируем модель**

Используем метрики:
- Accuracy Score
- F1 Micro 
- F1 Macro (точнее для несбалансированных классов)

Проверим точность модели на тестовой выборке, используя текущие веса модели 

In [92]:
# Валидируем модель на тестовой выборке

def validation():
    
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            
            # Для модели входные данные
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            
            # предсказание модели
            outputs = model(ids, mask, token_type_ids)
            
            # сохраняем данные в списках
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            
    return fin_outputs, fin_targets

In [93]:
# возврашаем предсказание (вероятность) и тру
outputs, targets = validation()

# Порог для принадлежности классов 0.5 для каждой категории 
outputs = np.array(outputs) >= 0.5

# оцениваем точность модели
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.9268659522466629
F1 Score (Micro) = 0.7845864117050557
F1 Score (Macro) = 0.6299690178109391


In [85]:
# test
pred = np.array([[0.2,0.2,0.5],[0.8,0.2,0.9]]) >= 0.5
targ = np.array([[0,1,1],[1,0,1]])
metrics.f1_score(pred,targ,average='micro')

0.8571428571428571